## The program does the following:
- Read the files (either .tiff or .jpg) from the given directory for ONE volume
- ALL the files in that volume are read in one go
- The files which throw errors and stop the code from running are stored in a seperate csv file, called <i>errors_year.csv</i> where year is the year from the volume, in the same directory.
- The crop coordinates for other files are stored in another csv, <i>year.csv</i>, in the same directory.
- Note: Some pages are not cropped but also don't throw errors. Those pages are not stored in the <i>errors_year.csv</i> file, but are rather stored in the <i>year.csv</i> file.

<b>Note: This file has been renamed. Previously, <i>test_functs.ipynb</i> </b>

In [ ]:
%run crop_functions_updated_.py

In [ ]:
from crop_functions_updated_ import *
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 50]
mod = importlib.reload(sys.modules['crop_functions_updated_'])  
vars().update(mod.__dict__)

In [ ]:
dirpath = "/Users/nitingupta/Desktop/OTB/MarginaliaRemoval/volumes/1898-JPG-testing"
content = os.listdir(dirpath)

# content_jpg = [x for x in os.listdir(dirpath) if '.jpg' in x]
# print(content_jpg)

In [ ]:
# Get the year
import re
fileName = re.findall(".+(\/.+)$", dirpath)  # for non-windows based paths
# fileName = re.findall(".+(\\.+)$", dirpath)  # for windows based paths
year = fileName[0][1:5]
year

In [ ]:
# changed the second argument from 10 to 1 if we are testing one page only, else 10
# ints = r.sample(range(0,len(content)), 10)

ints = list(range(0, len(content)))

In [ ]:
content = [content[i] for i in ints]
content.sort()

In [ ]:
fType = 'jpg' if "JPG" in dirpath else 'tiff'
sample = [dirpath+'/'+ x for x in content if fType in x and '~' not in x]
print(len(sample))

In [ ]:
## Pages which are throwing errors might need to cropped manually
# sample.remove('/Users/nitingupta/Desktop/OTB/MarginaliaRemoval/volumes/1902-JPG-iau-31858020993030-1647289160/00172.jpg')
# sample

In [ ]:
if (os.path.isfile('sample_csv.csv')):
    os.remove('sample_csv.csv')

In [ ]:
errorList = []  # will contain the list of all the file paths which throw errors
errorPath = ''
freq = 0
while True:
    freq += 1
    # Do make sure that the sample_csv.csv has been deleted (or is empty)
    # Because the method below will append to the csv if it already exists
    errorPath = crop2csv(sample, dil_iter=24)
    
    if (errorPath != ''):
        errorPathIndex = sample.index(errorPath)
        sample = sample[errorPathIndex+1:]
        errorList += [errorPath]
    else:
        break

In [ ]:
# Read the csv and removed the unwanted header information lines
df = pd.read_csv('sample_csv.csv')
df = df.drop(df[df.path == 'path'].index)

csvPath = './' + year + '.csv'
df.to_csv(csvPath, index_label=False, index=False)

os.remove('sample_csv.csv')

In [ ]:
# Create a dictionary of the errorList values
errorDict = {
    'year': int(year),
    'errorpaths':errorList}
# errorDict

In [ ]:
# import os.path

# convert the dictionary to a dataframe
df_errors = pd.DataFrame.from_dict(errorDict)

errorFile = "errors_" + year + ".csv"
# if the file already exists, then just append to it without the header
if (os.path.isfile(errorFile)):
    df_errors.to_csv(errorFile, index = False, header = False, mode='a')
else:
    df_errors.to_csv(errorFile, index = False, header = True)